# Data access with STAC

This section introduces [STAC](https://stacspec.org/), the SpatioTemporal Asset Catalog. STAC provides a standardized way to structure metadata about spatialotemporal data. The STAC community are building APIs and tools on top of this structure to make working with spatiotemporal data easier.

This notebook will give a brief introduction to the concepts defined by STAC, before moving on to using the STAC API easily find and load data.

## Introduction to STAC

Users of STAC will interact most often with **Collections** and **Items** (there's also **Catalogs**, which group together collections). A Collection is just a collection of items, plus some additional metadata like the license and summaries of what's available on each item. You can view the collections available on the Planetary Computer at https://planetarycomputer.microsoft.com/catalog. There's also [STAC Index](https://stacindex.org/), which maintains a list of public catalogs.

Let's load up the collection for Sentinel 2 Level 2-A, and compare it to the [HTML version](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a) on the Planetary Computer website (which is generated from the STAC metadata).

In [ ]:
import pystac

sentinel2 = pystac.read_file(
    "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-2-l2a/"
)
sentinel2

In [ ]:
print(sentinel2.description)

In [ ]:
sentinel2.extent.spatial.to_dict()

In [ ]:
sentinel2.extent.temporal.to_dict()

In [ ]:
sentinel2.to_dict()

Now let's take a look at a specific item. In this case, we'll load an item that covers the area just north of Buenos Aires: https://planetarycomputer-staging.microsoft.com/api/stac/v1/collections/sentinel-2-l2a/items/S2A_MSIL2A_20210811T135121_R024_T21HUC_20210812T044104

In [ ]:
item = pystac.read_file(
    "https://planetarycomputer-staging.microsoft.com/api/stac/v1/collections"
    "/sentinel-2-l2a/items/"
    "S2A_MSIL2A_20210811T135121_R024_T21HUC_20210812T044104"
)
item

The STAC item has a whole bunch of metadata about the Sentinel-2 Scene.

In [ ]:
item.datetime

In [ ]:
item.bbox

In [ ]:
from pystac.extensions.projection import ProjectionExtension

ProjectionExtension.ext(item).epsg

In [ ]:
from pystac.extensions.eo import EOExtension

EOExtension.ext(item).cloud_cover

The most important pieces of metadata are links to the actual data. These are available under "assets"

In [ ]:
item.assets

For example, we can use the "tilejson" URL, combined with `ipyleaflet` and the Item's geometry to quickly visualize the data.

In [ ]:
from ipyleaflet import Map, TileLayer, GeoJSON, FullScreenControl
import shapely.geometry
import requests

center = shapely.geometry.shape(item.geometry).centroid.bounds[:2][::-1]

m = Map(center=center, zoom=12)
layer = TileLayer(
    url=requests.get(item.assets["tilejson"].href).json()["tiles"][0],
)
m.add_layer(layer)
m.add_control(FullScreenControl())

m.scroll_wheel_zoom = True
m

### Signing for the Planetary Computer

If you try to open any of the data assets (e.g. "B02" for the blue band) you'll get a 404 error.

In [ ]:
import requests

r = requests.get(item.assets["B02"].href)
r.status_code

So while the STAC *metadata* is all public, *data* from the Planetary Computer is typically in private Blob Storage containers. But, you can access it anonymously by *signing* the item / asset / href.

In [ ]:
import rioxarray
import planetary_computer

ds = rioxarray.open_rasterio(planetary_computer.sign(item.assets["B02"].href))
ds

You can sign ItemCollections, Items, Assets, or URLs. Once a URL has been signed, it can be opened with any of your favorite tools (rioxarray, rasterio / GDAL, QGIS, etc.).

## Querying the STAC API

In the examples we've seen so far, we've just been given a STAC item. How do you find the items you want in the first place? That's where a STAC **API** comes in.

A STAC API is some web service that accepts queries and returns STAC objects. The ability to handle queries is what differentiates a STAC API from a *static* STAC catalog, where items are just present on some file system.

![image.png](ms-stac.png)

Visit https://planetarycomputer.microsoft.com/api/stac/v1/docs for documentation on a STAC API with HTTP requests. We'll use [pystac-client](https://pystac-client.readthedocs.io/en/latest/).

In [ ]:
import pystac_client

bbox = [-58.92, -34.81, -58.26, -34.18]
date_range = "2020-01-01/2020-12-31"

catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime=date_range,
)

In [ ]:
%time items = list(search.get_all_items())

In [ ]:
import shapely.geometry

item = items[0]
center = shapely.geometry.shape(item.geometry).centroid.bounds[:2][::-1]

m = Map(center=center, zoom=8)
layer = TileLayer(
    url=requests.get(item.assets["tilejson"].href).json()["tiles"][0],
)
m.add_layer(layer)
m.add_control(FullScreenControl())

m.scroll_wheel_zoom = True
m

Oops, that image is pretty cloudy.

In [ ]:
EOExtension.ext(item).cloud_cover

Fortunately, with the STAC API, it's possible to filter out images that are cloudier than some threshold.

In [ ]:
import pystac_client

bbox = [-58.92, -34.81, -58.26, -34.18]
date_range = "2020-01-01/2020-12-31"

catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
low_cloud_search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=bbox,
    datetime=date_range,
    query={"eo:cloud_cover": {"lt": 10}}  # Just return items with few clouds
)

In [ ]:
%time items = list(low_cloud_search.get_all_items())

In [ ]:
len(items)

In [ ]:
item = items[0]
center = shapely.geometry.shape(item.geometry).centroid.bounds[:2][::-1]

m = Map(center=center, zoom=9)
layer = TileLayer(
    url=requests.get(item.assets["tilejson"].href).json()["tiles"][0],
)
m.add_layer(layer)
m.add_control(FullScreenControl())
m.add_layer(GeoJSON(data=shapely.geometry.mapping(shapely.geometry.box(*bbox))))
m.scroll_wheel_zoom = True
m

### Loading STAC Items into xarray

Many remote sensing datasets are provided as collections of COGs. Each COG covers some area, and was taken at a specific time and a specific wavelength. It's very convienient to work with this as a 4-dimensional DataArray indexed by `(time, band, lon, lat)`. The `stackstac` library is a great way to build this type of DataArray.

In [ ]:
signed_items = [planetary_computer.sign(item).to_dict() for item in items]

In [ ]:
%%time
import stackstac

ds = stackstac.stack(signed_items, assets=["B02", "B03", "B04", "B08"])
ds

Oh that's a big DataArray. We aren't quite ready to tackle it yet, so let's narrow things down a bit.

In [ ]:
import stackstac

ds = stackstac.stack(
    signed_items,
    assets=["B02", "B03", "B04", "B08"],
    bounds_latlon=bbox,
    resolution=90
)
ds

In [ ]:
from distributed import Client

client = Client()
client

In [ ]:
ds = (
    ds.where(lambda x: x > 0)  # filter out nodata
      .compute()
      .ffill(dim="time")  # AOI spans two Scene boundaries
)

So we just

1. Queried the millions of Sentinel-2 Level 2-A scenes for just ones matching our requirements (location, time, cloudiness) using the STAC API
2. Assembled the matching scenes into a 4-d DataArray using stackstac
3. Loaded those into memory in parallel using Dask

In [ ]:
blue = ds.sel(band="B02")
green = ds.sel(band="B03")
red = ds.sel(band="B04")
nir = ds.sel(band="B08")

In [ ]:
ndvi = (nir - red) / (nir + red)

In [ ]:
import matplotlib.pyplot as plt

ndvi[1].plot.imshow(figsize=(12, 9), cmap="YlGn");

In [ ]:
ts = ndvi.mean(dim=["y", "x"])
ts.plot()

In [ ]:
low = ts.argmin().item()

In [ ]:
ndvi.isel(time=low).plot.imshow(figsize=(12, 9), cmap="YlGn");

In [ ]:
peak = ndvi.max(dim="time")
diff = (peak - ndvi)

In [ ]:
diff.isel(time=1).plot.imshow(figsize=(12, 9), cmap="YlGn", vmax=1);

In [ ]:
diff.isel(time=low).plot.imshow(figsize=(12, 9), cmap="YlGn", vmax=1);